# Evaluate model for Vietnamese Stock Env

In [2]:
import vectorbt as vbt
import gymnasium as gym
from stock_env.common.common_utils import create_performance, plot_trade_log_v2
from stock_env.envs import *
from stock_env.common.evaluation import evaluate_agent, play_an_episode
from stock_env.algos.agent import Agent
import torch as th

# Evaluation

## Evaluation on random period

In [3]:
envs = gym.vector.make("SP500-v0", num_envs=1, asynchronous=False)
agent = Agent(envs)
agent.load_state_dict(
    th.load("/Users/manbnm/stock-env/model/SP500-v0_ppo_2022-12-05_21-49-34.pth")
)
agent.eval()

Agent(
  (critic): Sequential(
    (0): Linear(in_features=70, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (actor_mean): Sequential(
    (0): Linear(in_features=70, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

AttributeError: 'Agent' object has no attribute 'device'

In [ ]:
mean, std = evaluate_agent(agent, envs, n_eval_episodes=10)
print(f"Mean reward: {mean:.2f} +/- {std: .2f}")

# Evaluation on specific stock

In [ ]:
envs = gym.vector.make("SP500-v0", num_envs=1, asynchronous=False)
# run model to get detailed information in the enviroment
info = play_an_episode(agent, envs)
df = info["final_info"][0]["final_history"]
df.head()

In [ ]:
returns = df.set_index("time")["portfolio_value"].pct_change()
create_performance(returns)

In [ ]:
plot_trade_log_v2(df)

# Compare with buy-and-hold strategy and feature strategy

In [ ]:
# portfolio value from hodling
df["pv_from_holding"] = vbt.Portfolio.from_holding(df.close, init_cash=20e3).value()

In [ ]:
fig = df["portfolio_value"].vbt.plot_against(df["pv_from_holding"])
fig.show()